In [1]:
pip install vega_datasets

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install vega_datasets
import pandas as pd
import altair as alt
from vega_datasets import data

# --- 1. Define File Names ---
file_suicide = 'suicide-rate-by-country-2025.csv'
print("--- Memulai script visualisasi Peta Dunia (Perbaikan) ---")

# --- 2. Load Data Peta ---
# Memuat data geografi (outline negara)
# Ini adalah data TopoJSON yang punya 'properties.name' (nama negara)
countries_geom = alt.topo_feature(data.world_110m.url, 'countries')

# --- 3. Load & Process Data Suicide ---
try:
    df_suicide_raw = pd.read_csv(file_suicide)
    print("File suicide rate berhasil dimuat.")
except Exception as e:
    print(f"Gagal memuat file: {e}")
    exit()

# Proses data suicide (ambil data 2021)
suicide_cols_to_melt = ['SuicideRateCountries_2021']
if suicide_cols_to_melt[0] not in df_suicide_raw.columns:
    print("Error: Kolom 'SuicideRateCountries_2021' tidak ditemukan.")
    exit()

df_suicide_melted = df_suicide_raw.melt(id_vars=['country'],
                                        value_vars=suicide_cols_to_melt,
                                        var_name='Suicide_Indicator',
                                        value_name='Suicide Rate')
df_suicide_melted['Suicide Rate'] = pd.to_numeric(df_suicide_melted['Suicide Rate'], errors='coerce')
# Penting: Hapus baris yang datanya NaN SETELAH konversi
df_suicide_2021 = df_suicide_melted.dropna(subset=['Suicide Rate'])
print("Data suicide 2021 selesai diproses.")


# --- 4. (DIHAPUS) ---
# Tidak perlu merge manual. Kita akan pakai transform_lookup.


# --- 5. Create Map Chart (Static Version) - DIPERBAIKI ---
try:
    chart_map = alt.Chart(countries_geom).mark_geoshape(
        stroke='black', # Garis batas antar negara
        strokeWidth=0.2
    ).project(
        type='equirectangular' # Tipe proyeksi peta
    ).encode(
        # Warnai negara berdasarkan Suicide Rate
        # 'na_color' adalah warna untuk negara yang tidak ada datanya
        color=alt.Color('Suicide Rate:Q',
                        scale=alt.Scale(range='heatmap', domain=[0, 30]), # Set skala warna kuning-merah
                        legend=alt.Legend(title="Suicide Rate (per 100k)"),
                        condition=alt.Condition("!isValid(datum['Suicide Rate'])", alt.value('#lightgray'))
                       ),
        tooltip=[
            alt.Tooltip('country:N', title='Country'),
            alt.Tooltip('Suicide Rate:Q', title='Suicide Rate', format='.2f')
        ]
    ).transform_lookup(
        lookup='properties.name', # Kunci di PETA (nama negara)
        from_=alt.LookupData(df_suicide_2021, 'country', ['Suicide Rate', 'country'])
    ).properties(
        title='Global Suicide Rate (2021)',
        width=800,
        height=450
    ).interactive() # Dibuat interaktif dengan tooltip

    # Simpan sebagai file JSON (sekarang interaktif)
    chart_file = 'slide2_map_INTERACTIVE.json'
    chart_map.save(chart_file)
    
    print(f"\n--- BERHASIL ---")
    print(f"Peta dunia interaktif berhasil disimpan ke: {chart_file}")

except Exception as e:
    print(f"Error membuat Peta: {e}")

--- Memulai script visualisasi Peta Dunia (Perbaikan) ---
File suicide rate berhasil dimuat.
Data suicide 2021 selesai diproses.
Error membuat Peta: module 'altair' has no attribute 'Condition'
